# Imports & Data Loading

In [125]:
import numpy as np
import pandas as pd

from functools import reduce

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [145]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


dictionary_df = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

for key, df in dictionary_df.items():
    dictionary_df[key] = df[(df['Donor Agency'] == 'Bundesministerium fnr Wirtschaftliche Zusammenarbeit und Entwicklung')]
    dictionary_df[key] = dictionary_df[key][dictionary_df[key]['Sector'] != 'Administrative Costs of Donors']
    
    

# Data Processing for Globe Stacked

In [148]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



## Combined

In [228]:
df_merged

,Sector,amount_2013,clim_rel_amount_2013,clim_adapt_amount_2013,clim_miti_amount_2013,non_miti_adapt_2013,non_clim_2013,amount_2014,clim_rel_amount_2014,clim_adapt_amount_2014,...,clim_adapt_amount_2021,clim_miti_amount_2021,non_miti_adapt_2021,non_clim_2021,amount_2022,clim_rel_amount_2022,clim_adapt_amount_2022,clim_miti_amount_2022,non_miti_adapt_2022,non_clim_2022
0,"I.1.a. Education, Level Unspecified",87.245749,6.281512,3.838428,2.443084,80.964237,80.964237,73.771448,5.642177,4.251977,...,0.870306,0.666370,250.891593,250.891593,265.014798,2.829080,1.364922,1.464158,262.185718,262.185718
1,I.1.b. Basic Education,153.930790,0.185664,0.139591,0.046073,153.745126,153.745126,169.047826,0.158667,0.112319,...,0.380428,0.181201,269.793665,269.793665,237.274430,0.830314,0.566782,0.263531,236.444117,236.444117
2,I.1.c. Secondary Education,110.570602,4.461417,3.382913,1.078504,106.109184,106.109184,123.680119,5.417927,3.629494,...,18.651157,6.372143,313.863098,313.863098,336.204882,24.097309,17.452876,6.644433,312.107573,312.107573
3,I.1.d. Post-Secondary Education,104.939416,10.369314,3.066040,7.303274,94.570102,94.570102,91.545807,3.192433,2.932056,...,2.036141,2.501237,95.873669,95.873669,107.771939,5.137190,1.959194,3.177997,102.634749,102.634749
4,"I.2.a. Health, General",95.765407,0.024584,0.000000,0.024584,95.740824,95.740824,86.811676,3.661154,3.661154,...,0.843957,0.654806,164.711021,164.711021,174.278158,0.641031,0.493282,0.147749,173.637127,173.637127
5,I.2.b. Basic Health,158.133749,0.792894,0.214570,0.578325,157.340855,157.340855,173.291741,0.252810,0.192620,...,5.559028,2.666179,450.047339,450.047339,1199.102079,22.426779,19.360655,3.066124,1176.675299,1176.675299
6,I.3. Population Policies/Programmes & Reproduc...,154.994846,1.018042,0.010408,1.007634,153.976803,153.976803,160.573929,1.003464,0.163170,...,2.332346,0.000000,169.786222,169.786222,179.901281,3.351427,3.351427,0.000000,176.549854,176.549854
7,I.4. Water Supply & Sanitation,340.989113,102.046666,75.801629,26.245037,238.942447,238.942447,349.651904,115.889128,88.690010,...,154.849054,45.855554,289.587016,289.587016,479.942386,190.660330,141.068684,49.591647,289.282056,289.282056
8,I.5.a. Government & Civil Society-general,791.300249,56.859578,31.586324,25.273254,734.440671,734.440671,887.065155,27.507943,15.276165,...,69.077470,76.983475,1241.964643,1241.964643,1145.696631,111.030814,57.492710,53.538104,1034.665817,1034.665817
9,"I.5.b. Conflict, Peace & Security",132.299555,0.465190,0.341959,0.123231,131.834365,131.834365,142.017188,0.390373,0.239275,...,25.567888,3.941753,367.004014,367.004014,369.056762,26.374849,21.798283,4.576567,342.681913,342.681913


In [229]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)


globe_df = globe_df.loc[[38]]

In [230]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
38,5115.784928,992.115751,6323.119826,1197.406246,5316.503434,1152.733316,6897.008022,1660.725243,7204.68697,1715.085188,8012.691583,1904.175131,8632.582927,2310.213481,10294.707282,2563.918782,10593.415129,2972.043199,11452.254162,3333.997494


In [231]:
years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - globe_df[clim_rel_amount_col]

# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_rel_amount_{year}' for year in years] + 
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')



# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(lambda x: 'Climate Finance' if 'clim_rel_amount' in x else 'Other Funds')

melted_df['Amount'] = melted_df['Amount'] * 1_000_000



melted_df.to_csv("../upload_data/globe_df.csv")

## Split

In [177]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')\
                   or col.startswith('clim_adapt_') or col.startswith('clim_miti_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)
globe_df = globe_df.loc[[142]]


years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_adapt_col = f'clim_adapt_amount_{year}'
    clim_miti_col = f'clim_miti_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - (globe_df[clim_adapt_col] + globe_df[clim_miti_col])


# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_adapt_amount_{year}' for year in years] +
                          [f'clim_miti_amount_{year}' for year in years] +
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')


# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(
    lambda x: 'Klimaanpassung Finanzierung' if 'clim_adapt_amount' in x 
    else ('Klimaschutz Finanzierung' if 'clim_miti_amount' in x 
    else 'Andere ODA')
)
melted_df['Amount'] = melted_df['Amounbt'] * 1_000_000




In [178]:
melted_df.to_csv("../upload_data/split_globe.csv")

read_df = pd.read_csv('../upload_data/split_globe.csv')


In [179]:
split_df = pd.read_csv('../upload_data/split_globe.csv')


fig = px.bar(split_df, x='Year', y='Amount', color='Type',
            title='Globale Finanzierungssummen',
            labels={'Amount': 'Finanzierungssumme ($)', 'Year': 'Jahr'},
            category_orders={'Type': ['Andere ODA','Klimaschutz Finanzierung', 'Klimaanpassung Finanzierung']},
            color_discrete_map={'Andere ODA': 'orange', 'Klimaschutz Finanzierung': 'green', 'Klimaanpassung Finanzierung': 'blue'}
            )# This ensures consistent color ordering

fig.update_layout(title_x=0.5)

fig.show()

# Data Processing For Sector Breakdown

In [203]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation', 'Sector']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby('Sector').sum()[['Amount','clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['non_miti_adapt'] =grouped_df['Amount'] - (grouped_df['clim_miti_amount'] + grouped_df['clim_adapt_amount'])
    grouped_df['non_clim'] =grouped_df['Amount'] - grouped_df['clim_rel_amount']
    
        
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'non_miti_adapt': f"non_miti_adapt_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'non_clim': f"non_clim_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    



In [204]:
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Sector', how='outer'), processed_grouped_dfs)


In [205]:
df_merged.to_csv('../upload_data/sector_analysis.csv')

df_merged = pd.read_csv('../upload_data/sector_analysis.csv')

In [218]:
list(range(2013,2023))

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [206]:
year = 2013

In [207]:
selected_year = [col for col in df_merged.columns if col.endswith(f'{year}')]
selected_year.append('Sector')
filtered_year_df = df_merged[selected_year]

In [209]:
clim_rel_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_rel_amount_{year}', title='Climate Related Amount by Sector')
clim_rel_fig.show()


In [213]:
non_clim_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'non_clim_{year}', title='Non Climate Related Amount by Sector')
non_clim_fig.show()


In [214]:
clim_adapt_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_adapt_amount_{year}', title='Climate Apaptation Amount by Sector')
clim_adapt_fig.show()


In [215]:
clim_miti_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_miti_amount_{year}', title='Climate Mitigation Amount by Sector')
clim_miti_fig.show()


In [9]:
dictionary_df['2015'][['Sector', 'SECTOR']]

,Sector,SECTOR
0,IV.1. General Environment Protection,41010
1,IV.1. General Environment Protection,41020
2,IV.2. Other Multisector,43010
3,IV.2. Other Multisector,43010
4,IV.2. Other Multisector,43030
...,...,...
12596,I.1.b. Basic Education,11230
12598,IV.2. Other Multisector,43010
12601,IV.2. Other Multisector,43010
12611,I.5.a. Government & Civil Society-general,15110
